In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### 이전 대화 내용을 기억하는 multi-turn Chain

In [3]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id = "sql_history", connection="sqlite:///sqlite.db"
)

In [4]:
# SQLChatMessageHisory객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달.
# session_id은 sqldb안에 column, connection에서 sqlite은 파일명. 내용은 한글이라 깨져보일 수 있다.
chat_message_history =SQLChatMessageHistory(
    session_id = "sql_history2", connection="sqlite:///sqlite.db" 
)

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.utils import ConfigurableFieldSpec
from langchain_core.runnables import RunnableWithMessageHistory

In [6]:
prompt= ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful Q&A assistant.'),
        MessagesPlaceholder(variable_name="chat_history"),
        ('human', '{question}')
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
chain = prompt | llm | StrOutputParser()

In [8]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name = user_id,
        session_id = conversation_id,
        connection="sqlite:///sqlite.db",

    )

In [9]:

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        default="",
        is_shared=True
    )
]

In [29]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key='question',
    history_messages_key="chat_history",
    history_factory_config=config_fields # 대화 기록 조회시 참고할 파라미터를 설정. config_field를 매밍해준다. 
    # get_chat_history함수는 원래를 인자를 session_id로 하나만 받기로 되어있는데, 
    # 인자의 이름을 변경하거나, 개수를 늘려주고 싶으면 ConfigurableFieldSpec를 정의하고, history_factory_config에 넣어줘야한다.

)

In [30]:
# 세션 아이디를 구성하는곳
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [32]:
chain_with_history.invoke({"question": "안녕? 내이름은 재호야. 나는 진원이랑 결혼할 사이야."}, config)

'안녕하세요, 재호님! 진원이와 결혼할 사이라고 하니 정말 축하드립니다! 두 분의 행복한 결혼 생활을 응원합니다. 결혼 준비는 어떻게 하고 계신가요? 궁금한 점이나 나누고 싶은 이야기가 있다면 언제든지 말씀해 주세요!'

In [33]:
chain_with_history.invoke({"question": "내가 누구랑 결혼한다고?"}, config)

'재호님께서 진원이와 결혼할 사이라고 말씀하셨습니다. 맞나요? 더 이야기하고 싶은 것이 있으면 언제든지 말씀해 주세요!'

In [34]:
config2 = {"configurable": {"user_id": "user2", "conversation_id": "conversation2"}}

In [35]:
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config2)

'당신의 이름을 제가 알 수 없어서 정확히 말씀드릴 수는 없습니다. 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!'

: 